In [ ]:
# Import R libraries
from rpy2.robjects.packages import importr
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri

## Choosing a CRAN Mirror
# utils = importr('utils')
# utils.chooseCRANmirror(ind=1)
## Installing required packages
# utils.install_packages('devtools')
## Import packages
# devtools = importr('devtools')
## Import R Functions
# install_github = devtools.install_github
## Get SharedForestBinary Model
# utils.install_packages("SharedForestBinary", type="source")
# install_github("theodds/SharedForestPaper/SharedForest")
## Import R packages
SharedForest, SharedForestBinary = importr('SharedForest'), importr('SharedForestBinary')
## Allow conversions between R and Python
pandas2ri.activate()

# Import Python libraries
import datetime
import os
import pickle
import shutil
import time
import numpy as np
import pandas as pd

from scipy.stats import norm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import matplotlib
import hyperopt
from hyperopt import tpe, Trials, hp, fmin, STATUS_OK, STATUS_FAIL
from collections import OrderedDict
from sklearn.metrics import r2_score

# Specify the column name and mapping for the labels we are trying to predict

train_path ="Data/cgiar_train.csv"
target_path ="Data/target/hours_worked_7_all.csv"



def create_train_test(data_path, label_path):
    # load features

    df = pd.read_csv(data_path, dtype={'msisdn': str})
    df.set_index('msisdn', inplace=True, drop=True)
    # load labels

    targets = pd.read_csv(label_path, dtype={"msisdn": str}, skiprows=[1]).set_index('msisdn', drop=True)


    column_name = ['gender', 'hours_worked']
  
    targets = targets[column_name]
    for c in column_name:
        targets['target_{}'.format(c)] = targets[c]
        targets.drop(c, axis=1, inplace=True)
    targets = targets.dropna().astype('str').agg('-'.join, axis=1).to_frame('target')

    # disregard unnecessary columns, map values to binary representation, and drop null values
    # drop columns that have more than a certain fraction (e.g. 50%) of null values
    threshold = 0.5
    df = df[[x for x in df.columns if (df[x].isna().sum() / df.shape[0] < threshold)]]

    # drop irrelevant columns and merge with labels
    df = df.drop(["SITE_ID", "NAME_1", "NAME_3"], axis=1)
    df = df.merge(targets, left_index=True, right_index=True, how="inner")
    
    

    # one-hot encode second distric names
    # one_hot = pd.get_dummies(df['NAME_2'])
    # fill missing values with zeros - empirically better than using mean or median
    data = df.fillna(0)
    print(data.head())

    X_train, X_test, y_train, y_test = train_test_split(data.drop('target', axis=1), data['target'], test_size=0.2,
                                                        random_state=42)
    y_train = y_train.str.split('-', expand=True)
    y_train.columns = column_name
    y_test = y_test.str.split('-', expand=True)
    y_test.columns = column_name

    for c in column_name:
        y_train[c] = pd.to_numeric(y_train[c], errors='coerce').astype(int)
        y_test.iloc[:, 0] = pd.to_numeric(y_test.iloc[:, 0], errors='coerce').astype(int)
        y_test.iloc[:, 1] = pd.to_numeric(y_test.iloc[:, 1], errors='coerce').astype(int)

    # scale features so that they have 0 mean and std equal to 1
    scaler = preprocessing.StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # scaling the continous indicators data
    y_train_ = scaler.fit_transform(pd.DataFrame(y_train.iloc[:, 1]))
    y_test_ = scaler.transform(pd.DataFrame(y_test.iloc[:, 1]))


    y_train.iloc[:, 1] = list(pd.DataFrame(y_train_)[0].values)
    y_test.iloc[:, 1] = list(pd.DataFrame(y_test_)[0].values)

    return X_train, y_train, X_test, y_test


def create_model_inputs(train_path, target_path):
    # Create train test data
    X_train, y_train, X_test, y_test = create_train_test(train_path, target_path)
    #y_test.iloc[:, 0]=y_test.iloc[:, 0].astype(float)
    y_test.iloc[:, 1] = y_test.iloc[:, 1].astype(float)
    #y_test['gender'].astype(float)
    #y_test['gender'].astype(float)

    # Convert to R dataframe
    W = ro.conversion.py2rpy(X_train)
    W_test = ro.conversion.py2rpy(X_test)

    # Split Y into delat1 and delta2
    delta1 = ro.conversion.py2rpy(y_train.iloc[:, 0])
    delta2 = ro.conversion.py2rpy(y_train.iloc[:, 1])

    return W, W_test, delta1, delta2, y_test


# Store variables


start = time.time()
def get_objective(W, W_test, delta1, delta2, y_test):
    def objective(space):
        """
        The objective contains the function to be optimized(shared forest model) .
        :param space:
        :return:
        """
        #opts = SharedForestBinary.Opts(num_burn=5000, num_thin=1, num_save=5000, num_print=10000)
        opts = SharedForest.Opts(num_burn=5000, num_thin=1, num_save=5000, num_print=1000)
        hypers = SharedForest.Hypers(X=W, Y=delta2, W=W, delta=delta1,
                                     alpha=space['alpha'],
                                     beta=int(space['beta']),
                                     gamma=space['gamma'],
                                     num_tree=int(space['num_tree']),
                                     var_tau=space['var_tau'],
                                     k=2,  ## Determines kappa
                                     k_theta=2,)

        sb = SharedForest.SharedBart(X=W,  # Training covariate matrix for continuous response
                                     Y=delta2,  # Training continuous response (standardized)
                                     W=W,  # Training covariate matrix for binary response (likely the same as X)
                                     delta=delta1,  # Training binary response
                                     X_test=W_test,  # Testing X
                                     W_test=W_test,  # Testing X
                                     hypers_=hypers, opts_=opts)



        # Helper Funtion
        def apply_fx(x):
            return np.random.binomial(p=norm.cdf(x), n=1)

        print('Getting delta1 predictions \n')
        # Get delta1 predictions
        delta_star1 = [apply_fx(x) for x in sb[8]]
        y_1 = pd.DataFrame(delta_star1).apply(lambda x: x.value_counts().index[0])
        #y_1['msisdn'] = y_test.index
        #print(y_1)
        y_1.to_csv("Data/output/delta1_hours_worked_7_all.csv", index=False)

        
        # Get delta2 predictions
        print("Getting Delta 2 prediction \n")
        delta_star2 = [apply_fx(x) for x in sb[6]]
        
        #y_2 = pd.DataFrame(delta_star2).apply(lambda x: x.value_counts().index[0])
        y_2 = pd.DataFrame(sb[6]).apply(lambda x: x.mean)
                                        
        
        y_2.to_csv("Data/output/delta2_hours_worked_7_all.csv",index=False)
        y_test.iloc[:, 1] = y_test.iloc[:, 1].astype(float)
       

        # Create F1 output score
        print("Create F1 output score \n")
        g_a = f1_score(y_test.iloc[:, 0], y_1, average='weighted')
        l_a = r2_score(y_test.iloc[:, 1], y_2) 


        hours = int((time.time() - start) // (60 * 60))
        mins = int(((time.time() - start) // 60) % 60)
        print('{} hours {} minutes have passed'.format(hours, mins))

        if np.isnan(g_a):
            status = STATUS_FAIL
        else:
            status = STATUS_OK
       
        return {'loss': 1-l_a,
                'status': status,
                'g_a': g_a,
                'l_a': l_a,
                'hyper': space}
    return objective


def summarize_trials(trials):
    results = trials.trials
    results = sorted(results, key=lambda x: -x['result']['l_a'])
    if results:
        print('Best: {}'.format(results[0]['result']))

    results = sorted(results, key=lambda x: -x['result']['l_a'])

    if results:
        print('Best test accuracy: {}'.format(results[0]['result']))


def optimize(objective, space, trials_fname=None, max_evals=1):
    if trials_fname is not None and os.path.exists(trials_fname):
        with open(trials_fname, 'rb') as trials_file:
            trials = pickle.load(trials_file)
    else:
        trials = Trials()

    best = fmin(objective,
         space=space,
         algo=tpe.suggest,
         trials=trials,
         max_evals=max_evals)

    if trials_fname is not None:
        temporary = '{}.temp'.format(trials_fname)
        with open(temporary, 'wb') as trials_file:
            pickle.dump(trials, trials_file)
        shutil.move(temporary, trials_fname)

    return trials


model_type = "hours_worked_7_all"
def main(max_evals):
    W, W_test, delta1, delta2, y_test = create_model_inputs(train_path, target_path)
    dtime = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    fname = 'Data/model/{}_model_results_{}.pickle'.format(model_type, dtime)

    objective = get_objective(W, W_test, delta1, delta2, y_test)
    # Optimization space
    space = OrderedDict([('alpha', hp.choice('alpha', range(1, 3, 1))),
                     ('beta', hp.choice('beta', range(1, 4, 1))),
                     ('gamma', hp.uniform('gamma', 0.5, 1.0)),
                     ('num_tree', hp.choice('num_tree', range(50, 200, 5))),
                     ('var_tau', hp.uniform('var_tau', 0.5, 1.0)),
                     ('k_theta', hp.choice('k_theta', range(0, 1, 1))),
                     ])
    trials = optimize(objective,
                      space,
                      trials_fname=fname,
                      max_evals=max_evals)

    summarize_trials(trials)

    return trials

trials_final = main(max_evals=1)


## Load the model tunned hyperameters and results

In [26]:

trials = pd.read_pickle("/home/nlubalo/Documents/ubuntu_shared_folder/CGIAR_Gender/hours_worked_7_all_model_results_20220530-215847.pickle")
trials.trials

[{'state': 2,
  'tid': 0,
  'spec': None,
  'result': {'loss': 1.0761794247904626,
   'status': 'ok',
   'g_a': 0.5316768025282873,
   'l_a': -0.07617942479046258,
   'hyper': {'alpha': 1,
    'beta': 1,
    'gamma': 0.883758612654199,
    'k_theta': 0,
    'num_tree': 55,
    'var_tau': 0.6349118680120148}},
  'misc': {'tid': 0,
   'cmd': ('domain_attachment', 'FMinIter_Domain'),
   'workdir': None,
   'idxs': {'alpha': [0],
    'beta': [0],
    'gamma': [0],
    'k_theta': [0],
    'num_tree': [0],
    'var_tau': [0]},
   'vals': {'alpha': [0],
    'beta': [0],
    'gamma': [0.883758612654199],
    'k_theta': [0],
    'num_tree': [1],
    'var_tau': [0.6349118680120148]}},
  'exp_key': None,
  'owner': None,
  'version': 0,
  'book_time': datetime.datetime(2022, 5, 30, 19, 58, 47, 779000),
  'refresh_time': datetime.datetime(2022, 5, 30, 20, 3, 21, 21000)},
 {'state': 2,
  'tid': 1,
  'spec': None,
  'result': {'loss': 1.0685427574807045,
   'status': 'ok',
   'g_a': 0.47609022556390